In [1]:
# Import required packages 
from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state=0
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
rng = np.random.RandomState(seed=random_state)
%matplotlib inline

In [2]:
path_data = os.path.join(os.getcwd(),'train_data.csv')
flights_train = pd.read_csv(path_data, delimiter = ',')
path_data = os.path.join(os.getcwd(),'test_data.csv')
flights_test = pd.read_csv(path_data, delimiter = ',')

In [3]:
X_train_full = flights_train.drop('ARR_DELAY_GROUP', axis=1).values.astype(np.float) # Training features
y_train_full = flights_train['ARR_DELAY_GROUP'].values # Training labels
X_test = flights_test.drop('ARR_DELAY_GROUP', axis=1).values.astype(np.float) # Training features
y_test = flights_test['ARR_DELAY_GROUP'].values # Training labels

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, 
                                                  test_size=0.2, random_state=random_state)

In [5]:
sc = StandardScaler().fit(X_train)
X_train_sc = sc.transform(X_train)
X_val_sc = sc.transform(X_val)
X_test_sc = sc.transform(X_test)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from scipy.stats import mode # Computes the mode of a signal
from sklearn.metrics import accuracy_score
names = [ "Random Forest","Naive Bayes"]
classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(n_neighbors=9),
    SVC(kernel="linear", probability=True, random_state=random_state),
    SVC(kernel='rbf', probability=True, random_state=random_state),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=50,random_state=random_state),
    MLPClassifier(random_state=random_state),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]
ca_score = {} # Classification accuracy
ce_score = {} # Cross-entropy
for name, clf in zip(names, classifiers):
    clf.fit(X_train_sc, y_train)
    ca_score[name] = clf.score(X_val_sc, y_val)
    ce_score[name] = log_loss(y_val, clf.predict_proba(X_val_sc))
print('Classification performance on validation set:')
for clf in names:
    print ("{}, accuracy: {:.3f}, log-loss: {:.3f}".format(clf, ca_score[clf], ce_score[clf]))

Classification performance on validation set:
Random Forest, accuracy: 0.551, log-loss: 1.171
Naive Bayes, accuracy: 0.416, log-loss: 6.069


In [7]:
from sklearn.dummy import DummyClassifier
# Your code goes here

# most_frequent strategy
dcl_mf = DummyClassifier(strategy='most_frequent')
dcl_mf.fit(X_train_sc, y_train) # Clf user guide for alternative strategy options
y_most_frequent = dcl_mf.predict(X_val)

# uniformly random prediction strategy
# Set random_state parameter to ensure reproducibility
dcl_rnd = DummyClassifier(strategy='uniform', random_state=10).fit(X_train_sc, y_train) 
y_random = dcl_rnd.predict(X_val)

print("Baseline classification accuracy on validation set (most frequent class): {:.3f}".
      format(accuracy_score(y_val, y_most_frequent)))
print("Baseline classification accuracy on validation set (random prediction): {:.3f}".
      format(accuracy_score(y_val, y_random)))

Baseline classification accuracy on validation set (most frequent class): 0.328
Baseline classification accuracy on validation set (random prediction): 0.082


In [9]:
pred_proba_dummy = dcl_mf.predict_proba(X_val_sc)
print("Log-loss scores:\nDummy classifier {}".
      format(log_loss(y_val, pred_proba_dummy)))

Log-loss scores:
Dummy classifier 23.204374141264378


In [10]:
from sklearn.metrics import cohen_kappa_score, make_scorer
kappa_scorer = make_scorer(cohen_kappa_score)

In [12]:
names = ["Decision Trees", "Random Forest"]
classifiers = [
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=50,random_state=random_state)]
ca_score = {} # Classification accuracy
ce_score = {} # Cross-entropy
kc_score = {}
for name, clf in zip(names, classifiers):
    clf.fit(X_train_sc, y_train)
    ca_score[name] = clf.score(X_val_sc, y_val)
    ce_score[name] = log_loss(y_val, clf.predict_proba(X_val_sc))
    kc_score[name] = cohen_kappa_score(y_val,clf.predict(X_val_sc))
print('Classification performance on validation set:')
for clf in names:
    print ("{}, accuracy: {:.3f}, log-loss: {:.3f}, kappa score: {:.3f}".format(clf, ca_score[clf], ce_score[clf], kc_score[clf]))

Classification performance on validation set:
Decision Trees, accuracy: 0.545, log-loss: 1.731, kappa score: 0.421
Random Forest, accuracy: 0.533, log-loss: 1.395, kappa score: 0.388


In [ ]:
names = ["RBF SVC", "Logistic Regression"]
classifiers = [SVC(kernel='rbf', probability=True, random_state=random_state), LogisticRegression()]
ca_score = {} # Classification accuracy
ce_score = {} # Cross-entropy
kc_score = {}
for name, clf in zip(names, classifiers):
    clf.fit(X_train_sc, y_train)
    ca_score[name] = clf.score(X_val_sc, y_val)
    ce_score[name] = log_loss(y_val, clf.predict_proba(X_val_sc))
    kc_score[name] = cohen_kappa_score(y_val,clf.predict(X_val_sc))
print('Classification performance on validation set:')
for clf in names:
    print ("{}, accuracy: {:.3f}, log-loss: {:.3f}, kappa score: {:.3f}".format(clf, ca_score[clf], ce_score[clf], kc_score[clf]))